# Import dependencies

In [1]:
import os
import re
import math
import pickle
import numpy as np
import pandas as pd
import importlib

import spacy

from utilities import _support
from utilities._support import ig_profiles

from nltk.tokenize import word_tokenize, sent_tokenize

importlib.reload(_support)

<module 'utilities._support' from 'c:\\Users\\tommaso\\Documents\\_Projects\\IGScraper\\utilities\\_support.py'>

# Load data

In [2]:
with open('utilities/lookup_users.pkl', 'rb') as f:
    full_lookup = pickle.load(f)

df = pd.concat((
        pd.read_json('gitshared_csv/giorgiameloni/giorgiameloni_posts-comment_20230319_0-1026_preprocessed__pt.0.json'),
        pd.read_json('gitshared_csv/giorgiameloni/giorgiameloni_posts-comment_20230319_0-1026_preprocessed__pt.1.json'),
        pd.read_json('gitshared_csv/giorgiameloni/giorgiameloni_posts-comment_20230319_0-1026_preprocessed__pt.2.json'),
    ))

In [3]:
df

,post_id,user,comment,comment_datetime,comment_likes,comment_prep,comment_token,comment_lemma,comment_postag
0,Ch7v-jBrUU2,user_0000141320,"Con la Lega e soprattutto con FI, la vedo molto dura. Molti italiani, non si fidano più. Troppi NO sono diventati SI.",2022-08-31T18:29:14.000Z,67,con la lega e soprattutto con fi la vedo molto dura. molti italiani non si fidano più. troppi no sono diventati si.,"[[con, la, lega, e, soprattutto, con, fi, la, vedo, molto, dura], [molti, italiani, non, si, fidano, più], [troppi, no, sono, diventati, si]]","[[con, la, legare, e, soprattutto, con, fi, la, vedere, molto, duro], [molto, italiano, non, si, fidare, molto], [troppo, no, essere, diventare, si]]","[[E, RD, S, CC, B, E, S, PC, V, B, A], [DI, S, BN, PC, V, B], [DI, BN, VA, V, PC]]"
1,Ch7v-jBrUU2,user_0000097699,E l'obbligo vaccinale??? Ce ne sono abbastanza di persone che hanno avuto episodi avversi GRAVI !!! RISPOSTA?? ATTENDIAMO,2022-08-31T18:22:18.000Z,42,e lobbligo vaccinale ce ne sono abbastanza di persone che hanno avuto episodi avversi gravi risposta attendiamo,"[[e, lobbligo, vaccinale, ce, ne, sono, abbastanza, di, persone, che, hanno, avuto, episodi, avversi, gravi, risposta, attendiamo]]","[[e, lobbligo, vaccinale, ce, ne, essere, abbastanza, di, persona, che, avere, avere, episodio, avversare, grave, rispondere, attendere]]","[[CC, S, A, PC, PC, V, B, E, S, PR, VA, V, S, A, A, S, V]]"
2,Ch7v-jBrUU2,user_0000080781,"Ma come cazzo fai a dire una roba del genere, hai votato a favore del green pass insieme a quel pagliaccio di Salvini",2022-08-31T18:58:37.000Z,14,ma come cazzo fai a dire una roba del genere hai votato a favore del green pass insieme a quel pagliaccio di salvini,"[[ma, come, cazzo, fai, a, dire, una, roba, del, genere, hai, votato, a, favore, del, green, pass, insieme, a, quel, pagliaccio, di, salvini]]","[[ma, come, cazzo, fare, a, diro, una, roba, del, genera, avere, votare, a, favore, del, green, pass, insieme, a, quello, pagliaccio, di, salvini]]","[[CC, E, S, V, E, V, RI, S, E_RD, S, VA, V, E, S, E_RD, SP, S, B, E, DD, S, E, SP]]"
3,Ch7v-jBrUU2,user_0000137237,"Ma per favore... Che quando era il momento una vera opposizione non c è stata, come non c è stata sulle restrizioni alla Russia e all invio di armi in Ucraina, motivo per cui prossimo inverno saremo nella merda che non ci scaldera'...",2022-08-31T19:02:35.000Z,10,ma per favore. che quando era il momento una vera opposizione non c è stata come non c è stata sulle restrizioni alla russia e all invio di armi in ucraina motivo per cui prossimo inverno saremo nella merda che non ci scaldera.,"[[ma, per, favore], [che, quando, era, il, momento, una, vera, opposizione, non, c, è, stata, come, non, c, è, stata, sulle, restrizioni, alla, russia, e, all, invio, di, armi, in, ucraina, motivo, per, cui, prossimo, inverno, saremo, nella, merda, che, non, ci, scaldera]]","[[ma, per, favore], [che, quando, essere, il, momento, una, vero, opposizione, non, c, essere, stare, come, non, c, essere, stare, sulla, restrizione, alla, russia, e, all, inviare, di, armare, in, ucraino, motivare, per, cui, prossimo, inverno, essere, nella, merda, che, non, ci, scaldera]]","[[CC, E, S], [PR, CS, V, RD, S, RI, A, S, BN, SP, VA, V, E, BN, SP, VA, V, E_RD, S, E_RD, SP, CC, E_RD, S, E, S, E, A, S, E, PR, A, S, V, E_RD, S, PR, BN, PC, V]]"
4,Ch7v-jBrUU2,user_0000129439,Vogliamo un reintegro immediato di tutti i lavoratori discriminati e/o licenziati. Inclusi ed urgentemente i dottori!!!,2022-08-31T21:36:20.000Z,4,vogliamo un reintegro immediato di tutti i lavoratori discriminati eo licenziati. inclusi ed urgentemente i dottori,"[[vogliamo, un, reintegro, immediato, di, tutti, i, lavoratori, discriminati, eo, licenziati], [inclusi, ed, urgentemente, i, dottori]]","[[volere, un, reintegrare, immediato, di, tutto, il, lavoratore, discriminare, eo, licenziato], [incluso, ed, urgentemente, il, dottore]]","[[VM, RI, S, A, E, T, RD, S, A, FB, V], [V, CC, B, RD, S]]"
...,...,...,...,...,..

# Word2Vec

## Utils

In [4]:
import scipy as scipy

from gensim.models import Word2Vec, LdaModel
from gensim.models.phrases import Phrases, Phraser
from gensim.corpora import Dictionary

"""
Word 2 Vec
"""

def w2v(sentences, model='SG', vector_size=300, min_count=10, get_bigrams=True, min_count_bigrams=10):
    if get_bigrams:
        sentences = generate_bi_grams(sentences, min_count=min_count_bigrams, threshold=0.5, progress_per=1000, scoring='npmi')

    w2v_model = Word2Vec(sentences, min_count=min_count, vector_size=vector_size, sg=(0 if model=='CBOW' else 1))
    return w2v_model

def get_w2v_vocab(w2v_model):
    return w2v_model.wv.key_to_index

def generate_bi_grams(sentences, **kwargs):
    phrases = Phrases(sentences, **kwargs)
    phrases_model = Phraser(phrases)
    return [phrases_model[sent] for sent in sentences]

# --------------------------------------------------------------------------- #

""" Word embeddings utils """

def wvec(wv_model, w):
    return wv_model.wv[w]

def wnorm(wv_model, w):
    vec = wv_model.wv[w] if type(w) is str else w
    return math.sqrt(sum([math.pow(v,2) for v in vec]))

def wsim(wv_model, pos, neg=[], topn=10, thresh=0, comp=-1):
    pos = [pos] if type(pos) is str else pos
    neg = [neg] if type(neg) is str else neg
    sim0 = list(wv_model.wv.most_similar(positive=pos, negative=neg, topn=topn))
    if thresh > 0:
        sim0 = [s for s in sim0 if s[1] >= thresh]
    if comp != -1:
        sim0 = [s[0] if comp==0 else s[1] for s in sim0]
    return sim0

def vsim(wv_model, vector, topn=10, comp=-1):
    sim = wv_model.wv.similar_by_vector(vector, topn=topn)
    if comp == 0:
        return [s[0] for s in sim]
    elif comp == 1:
        return [s[1] for s in sim]  
    else:
        return sim

def wanal(wv_model, str_anal, topn=10, thresh=0):
    p1 = str_anal[:str_anal.index('=')]
    p2 = str_anal[str_anal.index('=')+1:]
    neg = p1[:p1.index(':')].replace(' ', '')
    pos1 = p1[p1.index(':')+1:].replace(' ', '')
    pos2 = p2[:p2.index(':')].replace(' ', '')
    return wsim(wv_model, [pos1, pos2], neg, topn=topn, thresh=thresh)

def wdist(wv_model, w1, w2):
    w1 = wv_model.wv[w1] if type(w1) is str else w1
    w2 = wv_model.wv[w2] if type(w2) is str else w2
    return 1-scipy.spatial.distance.cosine(w1, w2)

def nwdist(wv_model, ws1, ws2):
    ws1 = [w for w in ws1 if w in wv_model.wv.key_to_index]
    ws2 = [w for w in ws2 if w in wv_model.wv.key_to_index]
    return wv_model.wv.n_similarity(ws1,ws2)

def wchoesion(wv_model, words):
    wsum = 0
    for w1 in words:
        for w2 in words:
            wsum = wsum + ((1-wdist(wv_model, w1, w2)) if w1!=w2 else 0)
    return (wsum / (math.pow(len(words),2)-len(words))) if (math.pow(len(words),2)-len(words))!=0 else 0

# Build model

In [5]:
# w2v needs list of sentences: list[list[string]]

sentences = [sentence for comment in df.comment_lemma for sentence in comment if sentence != [] and sentence!=None]

In [6]:
w2v_meloni = w2v(sentences, model='SG', vector_size=150, min_count=5, get_bigrams=True)
w2v_meloni_vocab = get_w2v_vocab(w2v_meloni)

collecting all words and their counts
PROGRESS: at sentence #0, processed 0 words and 0 word types
PROGRESS: at sentence #1000, processed 12026 words and 10768 word types
PROGRESS: at sentence #2000, processed 21355 words and 17546 word types
PROGRESS: at sentence #3000, processed 31831 words and 24317 word types
PROGRESS: at sentence #4000, processed 42548 words and 30571 word types
PROGRESS: at sentence #5000, processed 52931 words and 36067 word types
PROGRESS: at sentence #6000, processed 64719 words and 41884 word types
PROGRESS: at sentence #7000, processed 76178 words and 47417 word types
PROGRESS: at sentence #8000, processed 84815 words and 51219 word types
PROGRESS: at sentence #9000, processed 98464 words and 57573 word types
PROGRESS: at sentence #10000, processed 110517 words and 62853 word types
PROGRESS: at sentence #11000, processed 124155 words and 68713 word types
PROGRESS: at sentence #12000, processed 136778 words and 73767 word types
PROGRESS: at sentence #13000, p

In [73]:
wsim(w2v_meloni, 'destro', topn=100)

[('dx', 0.6598888635635376),
 ('estremo_destro', 0.6425057053565979),
 ('moderato', 0.6260298490524292),
 ('liberale', 0.6167868971824646),
 ('sinistro', 0.6165724396705627),
 ('centrodestra', 0.6118800640106201),
 ('storicamente', 0.6042223572731018),
 ('sputare_veleno', 0.6036906838417053),
 ('fidatevi', 0.6033162474632263),
 ('coeso', 0.5922285318374634),
 ('centrare_destro', 0.5919340252876282),
 ('lastensione', 0.5918657779693604),
 ('allestrema', 0.5902897715568542),
 ('lastensionismo', 0.5854584574699402),
 ('coalizione', 0.581679105758667),
 ('centrosinistra', 0.580664336681366),
 ('merdacce', 0.580007791519165),
 ('femminismo', 0.5788995027542114),
 ('lalternativa', 0.5786305665969849),
 ('figa', 0.5754082202911377),
 ('simpatizzare', 0.5754010677337646),
 ('fdl', 0.5752023458480835),
 ('democristiano', 0.574618935585022),
 ('centrodestra_unire', 0.5734031796455383),
 ('compattare', 0.5717905163764954),
 ('reazionario', 0.5710994601249695),
 ('unaccozzaglia', 0.570314586162567

In [28]:
wsim(w2v_meloni, 'pd', topn=100)

[('ms', 0.7272936701774597),
 ('pdioti', 0.6863657832145691),
 ('ms_.', 0.6817016005516052),
 ('grillini', 0.6777219772338867),
 ('dragone', 0.6768385171890259),
 ('centrosinistra', 0.6765967011451721),
 ('partitino', 0.6717366576194763),
 ('sinistro', 0.6676066517829895),
 ('stalla', 0.6613448858261108),
 ('sx', 0.6443790197372437),
 ('cinque_stella', 0.6431407332420349),
 ('pdidioti', 0.6408700346946716),
 ('pidioti', 0.6390442252159119),
 ('piddini', 0.6377652287483215),
 ('sinistroidi', 0.6357735395431519),
 ('p.d', 0.6280328035354614),
 ('leu', 0.6230994462966919),
 ('calenda_renzi', 0.6218611001968384),
 ('comunista', 0.6213914752006531),
 ('cattocomunista', 0.6204119324684143),
 ('pdiota', 0.6187993884086609),
 ('piddioti', 0.6176099181175232),
 ('antisistema', 0.6124513745307922),
 ('centrodestra', 0.609886109828949),
 ('cancro_dellitalia', 0.6092943549156189),
 ('calenda', 0.6070606708526611),
 ('mstalle', 0.6068483591079712),
 ('sboarina', 0.6067449450492859),
 ('lettamaio', 